In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import os
import json
import glob
import itertools
import operator

if not os.getcwd().endswith("src"):
    os.chdir("src")

from log_processing.process_logs import rank_results_by_predicate

if os.getcwd().endswith("src"):
    os.chdir("..")

In [ ]:
paths = [
    "models/sweep/a4_focal/base_512_focal",
    "models/sweep/a4_temperature/base_512_temperature",
    "models/sweep/a4_base/base_256",
    "models/sweep/a4_base/base_512",
    "models/sweep/a4_base/base_512_200",
    "models/sweep/a4_brier/base_512_brier",
]
sweep_configs = [
    "configs/sweep_configs/a4_focal/base_512_focal_sweep.json",
    "configs/sweep_configs/a4_temperature/base_512_temperature_sweep.json",
    "configs/sweep_configs/a4_base/base_256_sweep.json",
    "configs/sweep_configs/a4_base/base_512_sweep.json",
    "configs/sweep_configs/a4_base/base_512_200_sweep.json",
    "configs/sweep_configs/a4_brier/base_512_brier_sweep.json",
]
iterables = []
for sweep_config in sweep_configs:
    with open(sweep_config, "r") as f:
        sweep_config = json.load(f)
    keys = list(sweep_config.keys())
    values = list(sweep_config.values())
    iteration = list(itertools.product(*values))
    iterables.append(iteration)

In [27]:
criterion = operator.itemgetter("ndcg")
results = rank_results_by_predicate(
    paths=paths, j=-1, criterion=criterion, asc=False, read_metric_file_instead=True
)

for path, idx, hyperparameters, lines, line, criterion_value in results:
    idx = int(idx)
    config_idx = paths.index(path)
    sweep_iterable = iterables[config_idx]
    sweep_set = sweep_iterable[int(idx)]

    print_path = "/".join(path.split("/")[-2:])
    print(
        f"{print_path} {idx=} {criterion_value=:0.4f} {line['positive_scores']['ece']=:0.4f}"
    )

a4_base/base_512 idx=0 criterion_value=0.3482 line['positive_scores']['ece']=0.5100
a4_base/base_512_200 idx=0 criterion_value=0.3467 line['positive_scores']['ece']=0.5090
a4_temperature/base_512_temperature idx=0 criterion_value=0.3459 line['positive_scores']['ece']=0.4842
a4_temperature/base_512_temperature idx=1 criterion_value=0.3405 line['positive_scores']['ece']=0.4582
a4_focal/base_512_focal idx=3 criterion_value=0.3246 line['positive_scores']['ece']=0.5758
a4_temperature/base_512_temperature idx=2 criterion_value=0.3216 line['positive_scores']['ece']=0.4309
a4_focal/base_512_focal idx=1 criterion_value=0.3131 line['positive_scores']['ece']=0.7340
a4_focal/base_512_focal idx=2 criterion_value=0.3127 line['positive_scores']['ece']=0.5936
a4_focal/base_512_focal idx=0 criterion_value=0.3020 line['positive_scores']['ece']=0.7368
a4_focal/base_512_focal idx=7 criterion_value=0.2874 line['positive_scores']['ece']=0.5274
a4_focal/base_512_focal idx=5 criterion_value=0.2778 line['posit

In [36]:
criterion = lambda x: x["positive_scores"]["ece"]
results = rank_results_by_predicate(
    paths=paths, j=-1, criterion=criterion, asc=True, read_metric_file_instead=True
)

for path, idx, hyperparameters, lines, line, criterion_value in results:
    idx = int(idx)
    config_idx = paths.index(path)
    sweep_iterable = iterables[config_idx]
    sweep_set = sweep_iterable[int(idx)]

    print_path = "/".join(path.split("/")[-2:])
    print(
        f"{print_path} {idx=} "
        + f"ndcg= {line['ndcg']} "
        + f"ece={line['calibration_scores']['ece']:0.4f} "
        + f"pos_ece={line['positive_scores']['ece']:0.4f}\n"
        + f"\t[0.0,0.1] ece={line['piecewise_scores'][0]['ece']:0.4f} "
        + f"[0.1,0.5] ece={line['piecewise_scores'][1]['ece']:0.4f} "
        + f"[0.5,0.9] ece={line['piecewise_scores'][2]['ece']:0.4f} "
        + f"[0.9,1.0] ece={line['piecewise_scores'][3]['ece']:0.4f} "
    )

a4_base/base_256 idx=0 ndcg= 0.23100667873712608 ece=0.0276 pos_ece=0.0858
	[0.0,0.1] ece=0.0003 [0.1,0.5] ece=0.2630 [0.5,0.9] ece=0.7277 [0.9,1.0] ece=0.9862 
a4_temperature/base_512_temperature idx=3 ndcg= 0.2641719234173251 ece=0.0019 pos_ece=0.4218
	[0.0,0.1] ece=0.0003 [0.1,0.5] ece=0.2399 [0.5,0.9] ece=0.7051 [0.9,1.0] ece=0.9615 
a4_temperature/base_512_temperature idx=2 ndcg= 0.321592061532106 ece=0.0023 pos_ece=0.4309
	[0.0,0.1] ece=0.0003 [0.1,0.5] ece=0.2310 [0.5,0.9] ece=0.6941 [0.9,1.0] ece=0.9442 
a4_temperature/base_512_temperature idx=1 ndcg= 0.3404928479736013 ece=0.0024 pos_ece=0.4582
	[0.0,0.1] ece=0.0003 [0.1,0.5] ece=0.2279 [0.5,0.9] ece=0.6898 [0.9,1.0] ece=0.9327 
a4_temperature/base_512_temperature idx=0 ndcg= 0.34588396911773767 ece=0.0024 pos_ece=0.4842
	[0.0,0.1] ece=0.0004 [0.1,0.5] ece=0.2234 [0.5,0.9] ece=0.6822 [0.9,1.0] ece=0.9228 
a4_base/base_512_200 idx=0 ndcg= 0.3466766555608838 ece=0.0024 pos_ece=0.5090
	[0.0,0.1] ece=0.0004 [0.1,0.5] ece=0.2219 [0

In [38]:
criterion = lambda x: x["piecewise_scores"][3]["ece"]
results = rank_results_by_predicate(
    paths=paths, j=-1, criterion=criterion, asc=True, read_metric_file_instead=True
)

for path, idx, hyperparameters, lines, line, criterion_value in results:
    idx = int(idx)
    config_idx = paths.index(path)
    sweep_iterable = iterables[config_idx]
    sweep_set = sweep_iterable[int(idx)]

    print_path = "/".join(path.split("/")[-2:])
    print(
        f"{print_path} {idx=} "
        + f"ndcg= {line['ndcg']} "
        + f"ece={line['calibration_scores']['ece']:0.4f} "
        + f"pos_ece={line['positive_scores']['ece']:0.4f}\n"
        + f"\t[0.0,0.1] ece={line['piecewise_scores'][0]['ece']:0.4f} "
        + f"[0.1,0.5] ece={line['piecewise_scores'][1]['ece']:0.4f} "
        + f"[0.5,0.9] ece={line['piecewise_scores'][2]['ece']:0.4f} "
        + f"[0.9,1.0] ece={line['piecewise_scores'][3]['ece']:0.4f} "
    )

a4_focal/base_512_focal idx=0 ndcg= 0.3020002296407738 ece=0.0013 pos_ece=0.7368
	[0.0,0.1] ece=0.0005 [0.1,0.5] ece=0.1718 [0.5,0.9] ece=0.6045 [0.9,1.0] ece=0.7414 
a4_focal/base_512_focal idx=1 ndcg= 0.3130893670390136 ece=0.0012 pos_ece=0.7340
	[0.0,0.1] ece=0.0005 [0.1,0.5] ece=0.1628 [0.5,0.9] ece=0.6108 [0.9,1.0] ece=0.7455 
a4_focal/base_512_focal idx=5 ndcg= 0.27777983446824106 ece=0.0015 pos_ece=0.7059
	[0.0,0.1] ece=0.0005 [0.1,0.5] ece=0.1764 [0.5,0.9] ece=0.6180 [0.9,1.0] ece=0.7895 
a4_focal/base_512_focal idx=4 ndcg= 0.2676430796004531 ece=0.0016 pos_ece=0.7116
	[0.0,0.1] ece=0.0005 [0.1,0.5] ece=0.1835 [0.5,0.9] ece=0.6209 [0.9,1.0] ece=0.7904 
a4_focal/base_512_focal idx=2 ndcg= 0.31272470160907695 ece=0.0027 pos_ece=0.5936
	[0.0,0.1] ece=0.0004 [0.1,0.5] ece=0.2148 [0.5,0.9] ece=0.6663 [0.9,1.0] ece=0.8565 
a4_focal/base_512_focal idx=3 ndcg= 0.32462230800756825 ece=0.0027 pos_ece=0.5758
	[0.0,0.1] ece=0.0004 [0.1,0.5] ece=0.2161 [0.5,0.9] ece=0.6657 [0.9,1.0] ece=0.8